# Revisión de código para 2.2 One-sided Jacobi numerical aproximación

**Fecha:** 13 de Abril de 2020

**Responsable de revisión:** León Garay y Dorely Morales

**Código revisado**

In [1]:
indices <- function(n) {
  # Crea una lista de tamaño (n-1)n/2 con pares de índices de la siguiente
  #  manera: (1,2),..,(1,n),(2,3),..,(2,n),...,(n-1,n)
  # Args: 
  #    n: número entero postivo 
  #       se refiere al número de columnas
  #Returns:
  #    lista con pares de índices
    a <- NULL
    b <- NULL
    indices <- NULL
    for (i in 1:(n-1)){
    a <- append(a,rep(i,n-i))
    b <- append(b,seq(i+1,n))    
    }
    for(i in 1:round(n*(n-1)/2))
    indices[[i]] <- list(c(a[i], b[i]))
    indices
}

In [2]:
ortogonal <- function(u,v,TOL=10^-8){
  # Verifica si dos vectores son ortogonales, arrojando un 1 si lo es, y un 0 si no lo es.
  # Args: 
    # u, v como vectores de la misma dimensión.Y un valor real de tolerancia TOL(10^-8).
    # Nota: Se sugiere una TOL mayor a 10^-32.
  # Returns: 
    # Valor booleano 0 (no son ortongoales), 1 (son ortogonales)
    if ( norm(u,type ="2") < TOL | norm(v,type ="2") < TOL){
        ret<-0
    } else{ 
        
        if( (u%*%v)  /(norm(u,type ="2")*norm(v,type ="2")) < TOL   ){
    ret<-1
  }
  else{
    ret<-0
  }       
    }
  ret
}

In [3]:
signo<-function(x) {
  # Indica el signo de un número x
  # Args: 
  #    x (numeric): número a revisar
  # Returns:
  #    1 si el número es positivo o cero
  #    -1 si el número es negativo
  
  ifelse(x<0,-1,1)
  }

In [4]:
solver <- function(U,S,V,b){
    # Construye la solución de un sistema de ecuaciones a partir de matrices 
    # U, S, V, y vector b. Se asume que S es diagonal. 
    # Para ello resuelve S d = U^Tb, para construir x=Vd.
    # Notas:
    # 1) Se utilizó la función backsolve para resolver el sistema triangular.
    # 2) Al ser S diagonal, es indistinto si usar un solver para matrices traingulares inferiores o superiores.
    # Args: 
    #  	    U (mxm),V(nxn), S(mxn) matriz diagonal y b (m) un vector.
    # Returns: 
    #      x vector (m)
  d = backsolve(S, t(U)%*%b)
  x = V%*%d
  return(x)
}

In [5]:
svd_jacobi_aprox <- function(A,TOL,maxsweep){
    # Función que calcula la descomposición de una matriz A en sus componentes U, S V, 
    # utilizando el método de Jacobi para calcular la factorización SVD.De esta forma 
    # la matriz A queda descompuesta de la siguiente forma: A = U*S*t(V).
    # Args: 
    #    A (matriz): Matriz de entrada (nxm) de números reales a la que se le calculará la descomposición SVD.
    #    TOL (numeric): controla la convergencia del método, siendo un valor real de 10^-8 (sugerido en la nota 3.3.d.SVD)
    #    Nota: Se sugiere una TOL mayor a 10^-32.
    #    maxsweep (numeric): número máximo de sweeps,donde cada sweep consiste de un número máximo(nmax)
    #    de rotaciones; y en cada sweep se ortogonalizan 2 columnas.
    # Returns: 
    #   Lista con 3 elementos, donde el primer elemento representan a las matriz S(mxm) matriz diagonal,el segundo a la matriz U(nxm)
    #   y el tercero y último a la matriz V (mxm).En conjunto estas tres matrices componen la factorización SVD de la matriz de entrada A.
    
    #dimensiones
    n<-dim(A)[2] #numero de columnas
    m<-dim(A)[1] #numero de filas
    nmax<-n*(n-1)/2

    #inicialza valores del ciclo
    ak<-A
    vk<-diag(n)
    sig <- NULL
    uk <- ak
    num_col_ortogonal<-0
    k<-0

    while(k<=maxsweep & num_col_ortogonal<nmax){
    num_col_ortogonal<-0
    ind <- indices(n)
    for(i in 1:nmax){
      col_j<-ak[,ind[[i]][[1]][2]]
      col_i<-ak[,ind[[i]][[1]][1]]
    
      #comprueba ortogonalidad  
      if(ortogonal(col_i,col_j,TOL)==1){
        num_col_ortogonal<-num_col_ortogonal+1
      }
      else{
        #calcula coeficientes de la matriz
        a<-sum(col_i*col_i)
        b<-sum(col_j*col_j)
        c<-col_i%*%col_j
        
        if(c<TOL){break}
        #calcula la rotacion givens que diagonaliza
        epsilon<-(b-a)/(2*c)
        t<-signo(epsilon)/(abs(epsilon)+sqrt(1+epsilon**2))
        cs<-1/sqrt(1+t**2)
        sn<-cs*t
        
        #actualiza las columnas de la matriz ak
        for(l in seq(1,m)){
          temp<-ak[l,ind[[i]][[1]][1]]
          ak[l,ind[[i]][[1]][1]]<-cs*temp-sn*ak[l,ind[[i]][[1]][2]]
          ak[l,ind[[i]][[1]][2]]<-sn*temp+cs*ak[l,ind[[i]][[1]][2]]
        }
        
        #actualiza las columnas de la matriz vk
        for(l in seq(1,n)){
          temp<-vk[l,ind[[i]][[1]][1]]
          vk[l,ind[[i]][[1]][1]]<-cs*temp-sn*vk[l,ind[[i]][[1]][2]]
          vk[l,ind[[i]][[1]][2]]<-sn*temp+cs*vk[l,ind[[i]][[1]][2]]
        
        }
       }
  }
  k<-k+1
 }
    #Obtener sigma
        for(i in 1:n){
        sig<- append(sig,norm(ak[,i],type ="2"))
    }

    #Obtener U
    for(i in 1:n){
        if (sig[i]<TOL){
            uk[,i]<-0  
        } else{
        uk[,i] <- ak[,i]/sig[i]
        }
    }

    # Indices de sigma ordenada en forma decreciente para ordenar V,S,U
    index <- order(sig,decreasing = TRUE)
    vk <- vk[,index]
    S <- diag(sig[index])
    uk <- uk[,index]

    list(S = S, U = uk, V= vk)
 }   

In [6]:
sel_solver<-function(A,b,TOL=10**-8,maxsweep=20){
    #Función resuelve un sistema de ecuaciones lineales (SEL) utilizando la descomposición SVD
    #por medio del método de One-sided Jacobi 
    #El SEL es de la forma Ax=b
    # Args: 
    #    A (float): matriz de incógnitas del SEL
    #    b (float): vector de igualdada del sistema
    #    TOL (numeric): controla la convergencia del método
    #    maxsweep (int): número máximo de sweeps 
    #Returns: x (float): vector solución 

    svd<-svd_jacobi_aprox(A,TOL,maxsweep)
    x<-solver(svd$U,svd$S,svd$V,b)
    x
}

**1.Sobre la documentación del código/de la función**

¿Se encuentran presentes en la implementación los siguientes elementos? Por favor, ingrese explicaciones detalladas.

**a) Descripción concisa y breve de lo que hace el código/la función**

La función de encuentra bien documentada.

**b) Descripción de sus argumentos de entrada, su significado y rango de valores que pueden tomar**

Parcialmente, se sugiere complementar la descripción de los argumentos de entrada especificando que A tiene que ser una matriz de números reales de dimensión mxn.

**c) Descripción de los tipos de argumentos de entrada y de salida (por ejemplo, valores enteros, reales, strings, dataframe, matrices, etc)**

Sí se encuentra bien documentado.

**d) Descripción de la salida de la función, su significado y valores/objetos que deben regresar**

Sí se encuentra bien documentado.

**2. Cumplimiento de objetivos del código/de la función**

**a) ¿El código cumple los objetivos para los que fue diseñado?**

Sí dado que genera un vector x.

**b) ¿La salida de la función genera el valor necesario?**

Parcialmente, es necesario corregir algunos errores que ocurren cuando se introduce una matriz A de mxn con todos sus elementos aij iguales a un mismo valor constante. Adicionalmente, se necesita hacer optimizaciones a la función ya que para matrices A de tamaño mediano no encuentra la SVD.

**3. Pruebas**

Ocupe la presente sección para hacer diseño de pruebas variando los parámetros que recibe el código la función en diferentes rangos para evaluar su comportamiento y/o detectar posibles fallos


**Test 1**

**Objetivo del test 1:** Verificar que la función se comporta correctamente para una matriz nula A de mxn.

**Implementación del test 1:**

In [7]:
c5 <- rep(0,25)
b5 <- c(7,1,33,24,-49)
A5 <- matrix(c5, nrow=5, ncol=5, byrow = T)
c4 <- rep(0,16)
b4 <- c(15,-6,17,-7)
A4 <- matrix(c4, nrow=4, ncol=4, byrow = T)
c3 <- rep(0,9)
b3 <- c(2,9,-5)
A3 <- matrix(c3, nrow=3, ncol=3, byrow = T)
c2 <- rep(0,4)
A2 <- matrix(c2, nrow=2, ncol=2, byrow = T)
b2 <- c(2,9)

TOL<-10**-8
maxsweep<-20

print(A2)
print(A3)
print(A4)
print(A5)
#x_sel1<-sel_solver(A1,b1,TOL = 10**-15,maxsweep=10000)

     [,1] [,2]
[1,]    0    0
[2,]    0    0
     [,1] [,2] [,3]
[1,]    0    0    0
[2,]    0    0    0
[3,]    0    0    0
     [,1] [,2] [,3] [,4]
[1,]    0    0    0    0
[2,]    0    0    0    0
[3,]    0    0    0    0
[4,]    0    0    0    0
     [,1] [,2] [,3] [,4] [,5]
[1,]    0    0    0    0    0
[2,]    0    0    0    0    0
[3,]    0    0    0    0    0
[4,]    0    0    0    0    0
[5,]    0    0    0    0    0


In [8]:
x_sel2 <- sel_solver(A2,b2,TOL,maxsweep)
#x_sel3 <- sel_solver(A3,b3,TOL,maxsweep)
#x_sel4 <- sel_solver(A4,b4,TOL,maxsweep)
#x_sel5 <- sel_solver(A5,b5,TOL,maxsweep)

ERROR: Error in backsolve(S, t(U) %*% b): singular matrix in 'backsolve'. First zero in diagonal [1]


**Principales hallazos del test 1**

La función arroja un error pues existen infinitas soluciones que satisfacen A=U*S*V^T. Esto debido a que si tomamos la matriz S nula de mxn y cualesquiera matrices U de mxm y V de nxn ortogonales tenemos una solución.

**Test 2**

**Objetivo del test 2:** Verificar que la función se comporta correctamente para una matriz no nula A de mxn con entradas constantes.

**Implementación del test 2:**

In [9]:
cte <- 5
c5 <- rep(cte,25)
b5 <- c(7,1,33,24,-49)
A5 <- matrix(c5, nrow=5, ncol=5, byrow = T)
c4 <- rep(cte,16)
b4 <- c(15,-6,17,-7)
A4 <- matrix(c4, nrow=4, ncol=4, byrow = T)
c3 <- rep(cte,9)
b3 <- c(2,9,-5)
A3 <- matrix(c3, nrow=3, ncol=3, byrow = T)
c2 <- rep(cte,4)
A2 <- matrix(c2, nrow=2, ncol=2, byrow = T)
b2 <- c(2,9)

TOL<-10**-8
maxsweep<-20

print(A2)
print(A3)
print(A4)
print(A5)
#x_sel1<-sel_solver(A1,b1,TOL = 10**-15,maxsweep=10000)

     [,1] [,2]
[1,]    5    5
[2,]    5    5
     [,1] [,2] [,3]
[1,]    5    5    5
[2,]    5    5    5
[3,]    5    5    5
     [,1] [,2] [,3] [,4]
[1,]    5    5    5    5
[2,]    5    5    5    5
[3,]    5    5    5    5
[4,]    5    5    5    5
     [,1] [,2] [,3] [,4] [,5]
[1,]    5    5    5    5    5
[2,]    5    5    5    5    5
[3,]    5    5    5    5    5
[4,]    5    5    5    5    5
[5,]    5    5    5    5    5


In [10]:
x_sel2 <- sel_solver(A2,b2,TOL,maxsweep)
#x_sel3 <- sel_solver(A3,b3,TOL,maxsweep)
#x_sel4 <- sel_solver(A4,b4,TOL,maxsweep)
#x_sel5 <- sel_solver(A5,b5,TOL,maxsweep)

ERROR: Error in backsolve(S, t(U) %*% b): singular matrix in 'backsolve'. First zero in diagonal [2]


**Principales hallazos del test 2**

La función arroja un error pues existen infinitas soluciones que satisfacen A=U*S*V^T. Esto debido a que si tomamos la matriz S nula de mxn y cualesquiera matrices U de mxm y V de nxn ortogonales tenemos una solución.

**Test 3**

**Objetivo del test 3:** Verificar que la función se comporta correctamente para una matriz A de nxn con n = 2,3,4,5.

**Implementación del test 3:**

In [11]:
c5 <- c(2,-1,4,1,-1,-1,3,-2,-1,2,5,1,3,-4,1,3,-2,-2,-2,3,-4,-1,-5,3,-4)
b5 <- c(7,1,33,24,-49)
A5 <- matrix(c5, nrow=5, ncol=5, byrow = T)
c4 <- c(1,-2,2,-3,3,4,-1,1,2,-3,2,-1,1,1,-3,-2)
b4 <- c(15,-6,17,-7)
A4 <- matrix(c4, nrow=4, ncol=4, byrow = T)
c3 <- c(2,-1,1,3,1,-2,-1,2,5)
b3 <- c(2,9,-5)
A3 <- matrix(c3, nrow=3, ncol=3, byrow = T)
c2 <- c(2,-3,5,2)
A2 <- matrix(c2, nrow=2, ncol=2, byrow = T)
b2 <- c(2,9)

TOL<-10**-8
maxsweep<-20

print(A2)
print(A3)
print(A4)
print(A5)
#x_sel1<-sel_solver(A1,b1,TOL = 10**-15,maxsweep=10000)

     [,1] [,2]
[1,]    2   -3
[2,]    5    2
     [,1] [,2] [,3]
[1,]    2   -1    1
[2,]    3    1   -2
[3,]   -1    2    5
     [,1] [,2] [,3] [,4]
[1,]    1   -2    2   -3
[2,]    3    4   -1    1
[3,]    2   -3    2   -1
[4,]    1    1   -3   -2
     [,1] [,2] [,3] [,4] [,5]
[1,]    2   -1    4    1   -1
[2,]   -1    3   -2   -1    2
[3,]    5    1    3   -4    1
[4,]    3   -2   -2   -2    3
[5,]   -4   -1   -5    3   -4


In [12]:
x_sel2 <- sel_solver(A2,b2,TOL,maxsweep)
x_sel3 <- sel_solver(A3,b3,TOL,maxsweep)
x_sel4 <- sel_solver(A4,b4,TOL,maxsweep)
x_sel5 <- sel_solver(A5,b5,TOL,maxsweep)

Buscamos que la función nos regrese aproximadamente $b2 = (2,9) $

In [13]:
A2%*%x_sel2

2
9


Podemos ver que funcionó correctamente, ya que Ax=b.

Ahora probaremos con $A_{3x3}$ buscamos $b3 = (2,9,-5)$

In [14]:
A3%*%x_sel3

2.327966
11.280633
-8.629119


En este caso podemos ver que la función no arrojó correctamente los valores de Ax=b.

Seguimos con el caso $A_{4x4}$ y buscamos que $b4 = (15,-6,17,-7)$

In [15]:
A4%*%x_sel4

22.127278
-12.551705
23.677302
-8.309975


Podemos observar que tampoco arrojó balores cercanos de Ax=b.

Finalmente probaremos con $A-{5x5}$ y buscamos $b5 = (7,1,33,24,-49)$

In [16]:
A5%*%x_sel5

1.013013
10.506870
62.822134
39.049742
-84.244898


Podemos observar que tampoco arrojó valores correctos.

**Principales hallazos del test 3**

La función arroja valores lejamos a los esperados para matrices con dimensiones mayores a 2x2.

**Test 4**

**Objetivo del test 4:** Verificar que la función se comporta correctamente para una matriz A de mxn con n = 2,3,4,5 y m = 2,3,4.

**Implementación del test 4:**

In [17]:
c5 <- c(2,-1,4,1,-1,-1,3,-2,-1,2,5,1,3,-4,1,3,-2,-2,-2,3)
b5 <- c(7,1,33,24,-49)
A5 <- matrix(c5, nrow=5, ncol=4, byrow = T)
c4 <- c(1,-2,2,-3,3,4,-1,1,2,-3,2,1)
b4 <- c(15,-6,17,-7)
A4 <- matrix(c4, nrow=4, ncol=3, byrow = T)
c3 <- c(2,-1,1,3,1,-2)
b3 <- c(2,9,-5)
A3 <- matrix(c3, nrow=3, ncol=2, byrow = T)
c2 <- c(2,-3,5,2)
A2 <- matrix(c2, nrow=2, ncol=2, byrow = T)
b2 <- c(2,9)

TOL<-10**-8
maxsweep<-20

print(A2)
print(A3)
print(A4)
print(A5)
#x_sel1<-sel_solver(A1,b1,TOL = 10**-15,maxsweep=10000)

     [,1] [,2]
[1,]    2   -3
[2,]    5    2
     [,1] [,2]
[1,]    2   -1
[2,]    1    3
[3,]    1   -2
     [,1] [,2] [,3]
[1,]    1   -2    2
[2,]   -3    3    4
[3,]   -1    1    2
[4,]   -3    2    1
     [,1] [,2] [,3] [,4]
[1,]    2   -1    4    1
[2,]   -1   -1    3   -2
[3,]   -1    2    5    1
[4,]    3   -4    1    3
[5,]   -2   -2   -2    3


In [18]:
x_sel2 <- sel_solver(A2,b2,TOL,maxsweep)
x_sel3 <- sel_solver(A3,b3,TOL,maxsweep)
x_sel4 <- sel_solver(A4,b4,TOL,maxsweep)
x_sel5 <- sel_solver(A5,b5,TOL,maxsweep)

Buscamos que la función nos regrese aproximadamente $b2 = (2,9) $

In [19]:
A2%*%x_sel2

2
9


Podemos ver que funcionó correctamente, ya que Ax=b.

Ahora probaremos con $A_{3x3}$ buscamos $b3 = (2,9,-5)$

In [20]:
A3%*%x_sel3

0.1666667
8.8333333
-3.6666667


En este caso podemos ver que la función no arrojó correctamente los valores de Ax=b.

Seguimos con el caso $A_{4x4}$ y buscamos que $b4 = (15,-6,17,-7)$

In [21]:
A4%*%x_sel4

19.2029412
-5.6382353
0.5911765
-11.7852941


Podemos observar que tampoco arrojó balores cercanos de Ax=b.

Finalmente probaremos con $A-{5x5}$ y buscamos $b5 = (7,1,33,24,-49)$

In [22]:
A5%*%x_sel5

27.976974
11.503845
18.094858
8.093239
-37.052412


Podemos observar que tampoco arrojó valores esperados.

**Principales hallazos del test 4**

La función arroja valores lejamos a los esperados para matrices con dimensiones mayores a 2x2.


**4. Resumen detallado de posibles puntos faltantes en implementación**

* Se sugiere añadir complementar la documentación con más información sobre la descripción de la función, args y returns.
* Necesita revisarse el caso de matrices con todos sus elementos iguales a una constante pues no encuentra la SVD.
* Es necesario mejorar el rendimiento de la función a partir del while que se encuentra en la línea 24 pues para matrices A de tamaño mediano: $10^4 \times 10^4$ la función no devuelve la lista con las matrices U,S,V.

**Sugerencias para resolver los puntos anteriores**

Se propone ajustar la función con respecto a cada uno de los puntos mencionados de la siguiente forma: